In [1]:
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import astropy.constants as const
from scipy.spatial import KDTree
from astropy.io import fits
from astropy.coordinates import SkyCoord
import regions
from regions import Regions

In [2]:
cd ..

/blue/adamginsburg/savannahgramze/cloudc/lactea-filament/lactea-filament


In [3]:
from jwst_plots import make_cat_use
from jwst_plots import JWSTCatalog
import cutout_manager as cm
import extinction as ex
import moment_maker as mm

In [4]:
cd notebooks

/blue/adamginsburg/savannahgramze/cloudc/lactea-filament/lactea-filament/notebooks


In [5]:
pos = SkyCoord('17:46:20.6290029866', '-28:37:49.5114204513', unit=(u.hour, u.deg))
l = 113.8*u.arcsec
w = 3.3*u.arcmin
reg = regions.RectangleSkyRegion(pos, width=l, height=w)

# CO

In [ ]:
fn_12CO = '/orange/adamginsburg/cmz/nobeyama/12CO-2.BEARS.FITS'
fn_13CO = '/orange/adamginsburg/cmz/nobeyama/13CO-2.FOREST.FITS'
fn_C18O = '/orange/adamginsburg/cmz/nobeyama/C18O-2.FOREST.FITS'

In [ ]:
cube_12CO = SpectralCube.read(fn_12CO, format='fits')
cube_13CO = SpectralCube.read(fn_13CO, format='fits')
cube_C18O = SpectralCube.read(fn_C18O, format='fits')

## Strong et al 1988 X-Factor

In [ ]:
subcube_12CO = cube_12CO.subcube_from_regions([reg])
subcube_12CO = subcube_12CO.with_spectral_unit(u.km/u.s).spectral_slab(-57*u.km/u.s, -53*u.km/u.s)#(-56*u.km/u.s, -54*u.km/u.s)#

subcube_12CO_mom0 = subcube_12CO.moment0()

In [ ]:
ax = plt.subplot(projection=subcube_12CO_mom0.wcs)
im = ax.imshow(subcube_12CO_mom0.value, origin='lower', cmap='inferno')
plt.colorbar(im, ax=ax, label=subcube_12CO_mom0.unit)

In [ ]:
ww = subcube_12CO_mom0.wcs
subcube_N = np.nansum(subcube_12CO_mom0.value) * subcube_12CO_mom0.unit * 2.3 * 10**20 * u.cm**-2 * u.K**-1 * u.km**-1 * u.s
pixel_area_physical = (ww.proj_plane_pixel_scales()[0] * 5 * u.kpc).to(u.cm, u.dimensionless_angles())**2
mass_Xfactor = (subcube_N * pixel_area_physical * 2.8 * u.u).to(u.Msun)
mass_Xfactor

## Isotopologue Masses

In [ ]:
def get_13CO_10_N(Tex, I, Ju=1):
    mu = (1.1046*10**(-19) * u.esu * u.cm).to(u.cm*(u.cm**(3/2) * u.gram**(1/2) * u.second**(-1)))
    B = 55101.011 * u.MHz
    Q_rot = get_Qrot(Tex, B)
    S = get_S(Ju)
    Ri = 1
    f = 1
    jpl = query_JLP("29001 C-13-O")
    freq = jpl['FREQ'].to(u.GHz)[Ju-1]
    splat = query_splat(' 13CO ')
    Eu = splat['upper_state_energy_K'] * u.K
    gu = jpl['GUP'][Ju-1]

    N = (3 * const.h * Q_rot)/(8 * np.pi**3 * S * mu**2 * Ri * gu) 
    N *= np.exp(Eu/(Tex))/(np.exp(const.h*freq/(const.k_B*Tex))-1) 
    N *= (f * (planck(Tex, freq)-planck(2.73*u.K, freq)))**(-1)
    N = N*I

    return N.to(u.cm**(-2))

In [ ]:
def get_C18O_10_N(Tex, I, Ju=1):
    mu = (1.1079*10**(-19) * u.esu * u.cm).to(u.cm*(u.cm**(3/2) * u.gram**(1/2) * u.second**(-1)))
    B = 54891.420 * u.MHz
    Q_rot = get_Qrot(Tex, B)
    S = get_S(Ju)
    Ri = 1
    f = 1
    jpl = query_JLP("29001 CO-18")
    freq = jpl['FREQ'].to(u.GHz)[Ju-1]
    splat = query_splat(' C18O ')
    Eu = splat['upper_state_energy_K'] * u.K
    gu = jpl['GUP'][Ju-1]

    N = (3 * const.h * Q_rot)/(8 * np.pi**3 * S * mu**2 * Ri * gu) 
    N *= np.exp(Eu/(Tex))/(np.exp(const.h*freq/(const.k_B*Tex))-1) 
    N *= (f * (planck(Tex, freq)-planck(2.73*u.K, freq)))**(-1) 
    N = N*I

    return N.to(u.cm**(-2))

In [ ]:
subcube_13CO = cube_13CO.subcube_from_regions([reg])
subcube_13CO = subcube_13CO.with_spectral_unit(u.km/u.s).spectral_slab(-57*u.km/u.s, -53*u.km/u.s)#(-56*u.km/u.s, -54*u.km/u.s)#

subcube_C18O = cube_C18O.subcube_from_regions([reg])
subcube_C18O = subcube_C18O.with_spectral_unit(u.km/u.s).spectral_slab(-57*u.km/u.s, -53*u.km/u.s)#(-56*u.km/u.s, -54*u.km/u.s)#

subcube_13CO_mom0 = subcube_13CO.moment0()
subcube_C18O_mom0 = subcube_C18O.moment0()

In [ ]:
column_density_13CO = get_13CO_10_N(20*u.K, subcube_13CO_mom0)
column_density_C18O = get_C18O_10_N(20*u.K, subcube_C18O_mom0)

In [ ]:
co_abundance = 10**(-4)

In [ ]:
ratio_12C_13C = 53 # from Wilson & Rood 1994
ww = wcs.WCS(subcube_13CO_mom0.header)
N_13CO = np.nansum(column_density_13CO.value) * u.cm**(-2)
pixel_physical_area = (ww.proj_plane_pixel_scales()[0] * 5 * u.kpc).to(u.cm, u.dimensionless_angles())**2
M_13CO = N_13CO * pixel_physical_area * 2.8 * u.Da
mass_13CO = M_13CO.to(u.M_sun) *ratio_12C_13C / co_abundance

In [ ]:
ratio_16O_18O = 327 # from Wilson & Rood 1994
ww = wcs.WCS(subcube_C18O_mom0.header)
N_C18O = np.nansum(column_density_C18O.value) * u.cm**(-2)
pixel_physical_area = (ww.proj_plane_pixel_scales()[0] * 5 * u.kpc).to(u.cm, u.dimensionless_angles())**2
M_C18O = N_C18O * pixel_physical_area * 2.8 * u.Da
mass_C18O = M_C18O.to(u.M_sun) * ratio_16O_18O / co_abundance

# Extinction Mass

In [ ]:
ext_map = ex.make_extinction_map(ext=CT06_MWGC(), Av_fill=85)
mass_extinction = ex.get_mass_estimate(ext_map, ww)
mass_extinction

# Mass Table

In [ ]:
methods = ['X-factor', '$^{13}$CO LTE', 'C$^{18}$O', 'A$_V \times 2e21']
masses = [mass_Xfactor, mass_13CO, mass_C18O, mass_extinction]